# NLP classification - supervised learning

In this example, you will learn how you can use supervised learning algorithms for NLP classification. We will use documents from mtsamples again. The task is to classify a document into its clinical specialty, e.g. pediatrics or hematology.

We will use classification algorithms as implemented in sci-kit learn, and evaluate with cross-validation before testing on unseen test data.

We will experiment with different ways of representing the documents for the classifiers.

material in parts from https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

Written by Sumithra Velupillai, March 2019 - updated February 2020

# 1: Packages
We will use a number of different packages for this exercise

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
try:
    import xlrd
except ImportError as e:
    !pip install xlrd
    import xlrd

import numpy as np

import warnings; warnings.simplefilter('ignore')


In [ ]:
# We'll use scikit-learn for the classification algorithms.
# https://scikit-learn.org/stable/

#from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [ ]:
## sklearn also has some nice funtions for representations

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## and for evaluation
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score



In [ ]:
## Since we're working with text, we might need to tokenize for some of these representations. 
# We'll use nltk here, but there are other nlp packages available for this
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords







In [ ]:
# You have also learnt about embedding representations. These can also be used for classification.
# We will use a library called Zeugma, which allows using pre-trained embedding models
#Zeugma library: https://github.com/nkthiebaut/zeugma

try:
    from zeugma.embeddings import EmbeddingTransformer
except ImportError as e:
    !pip install zeugma
    !pip install theano
    from zeugma.embeddings import EmbeddingTransformer

from datetime import datetime
print(datetime.now())

# 2: Corpus
Read in the training data.

In [ ]:

xlds_training = 'https://github.com/KCL-Health-NLP/nlp_examples/blob/master/classification/classification_trainingdata.xlsx?raw=true'
trainingdata = pd.read_excel(xlds_training)



Take a look at the content of the training data. What are we trying to classify? What are the labels we want to try to learn? How many instances do we have?

In [ ]:
trainingdata['label'].value_counts()

What types of features do you think would be useful for the classification task? Where can we get them? Take a look at one or two of the documents. Can you guess which classification label these belong to?

In [ ]:
trainingtxt_example = trainingdata['txt'].tolist()[0]
print(trainingtxt_example)

In [ ]:
trainingtxt_example = trainingdata['txt'].tolist()[231]
print(trainingtxt_example)

# 3: Representation - BoW

The most common baseline feature representation for text classification tasks is to use the *bag-of-words* representation, in a document-term matrix. Let's build a simple one using raw counts and only keeping a maximum of 500 features. We can use the CountVectorizer function from sklearn, and tokenize using a function from nltk.

In [ ]:
first_vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=None,
                             tokenizer=word_tokenize, max_features=500)
first_vectorizer.fit(trainingdata['txt'].tolist())
first_fit_transformed_data = first_vectorizer.fit_transform(trainingdata['txt'])


We can now look at this transformed representation for an example document.

In [ ]:
first_transformed_data = first_vectorizer.transform([trainingdata['txt'].tolist()[231]])
print (first_transformed_data)

What word is represented by the different indices? Have a look at a few examples.

In [ ]:
print (first_vectorizer.get_feature_names_out()[32])

In [ ]:
print(first_fit_transformed_data.shape)
print ('Amount of Non-Zero occurences: ', first_fit_transformed_data.nnz)

# 4: Classification
Let's build a classifier with this feature representation. In text classification, many classification algorithms have been shown to work well. Sci-kit learn has implementations for many different types of classification algorithms - have a look at their website!

Let's try a K nearest neighbour classifier.


In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(first_fit_transformed_data, trainingdata['label'])

We now have a trained model. But how do we know how well it works? Let's evaluate it on the test data.

In [ ]:

xlds_test = 'https://github.com/KCL-Health-NLP/nlp_examples/blob/master/classification/classification_test_data.xlsx?raw=true'
testdata = pd.read_excel(xlds_test)



## We need to transform this data to the same representation
first_fit_transformed_testdata = first_vectorizer.transform(testdata['txt'])

In [ ]:
first_fit_transformed_testdata
kneighbour_predicted = kneighbour_classifier.predict(first_fit_transformed_testdata)
kneighbour_predicted

Let's make a list of all the labels in our dataset to evaluate, and then run some standard evaluation metrics

In [ ]:
labels = list(set(testdata['label']))
print(metrics.classification_report(testdata['label'], kneighbour_predicted, target_names=labels))



What do you think about these results? There are probably ways of improving this, by changing the representation or maybe trying a different classifier model. 
__There is one main problem though: we can't use this test data to try different configurations! Why?__

# 5: N-fold cross-validation

We can employ n-fold cross-validation on the training data to experiment with different representations, parameters, and classifiers.

There are also various metrics that can be used to evaluate classification results.


In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(first_fit_transformed_data, trainingdata['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, first_fit_transformed_data, trainingdata['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, count vectorizer')
plt.show()

What happens if we try another classifier? Let's try a random forest classifier.

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0).fit(first_fit_transformed_data, trainingdata['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(rf_classifier, first_fit_transformed_data, trainingdata['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('Random forest, count vectorizer')
plt.show()

Was this better or worse? Are there any parameters worth changing?

# 6: Another representation model: Tf-idf
We have used a very simple bag-of-words representation. What happens if we try something else? Let's try tf-idf. This is considered a strong baseline in many text classification tasks.

In [ ]:

stopWords = list(stopwords.words('english'))
tfidf_vect = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopWords)
tfidf_vect.fit(trainingdata['txt'])
second_fit_transformed_data =  tfidf_vect.transform(trainingdata['txt'])
second_fit_transformed_data

What other parameters can you change in this representation? How does this look different from the CountVectorizer representation?

Let's now use this with the KNN classifier.

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(second_fit_transformed_data, trainingdata['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, second_fit_transformed_data, trainingdata['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, tf-idf vectorizer')
plt.show()

This looks better, doesn't it? Why do you think this works better?

# 7: Representations: embeddings

Last week, you learnt about embedding representations. What might be the benefit of using this type of representation instead of counts or tf-idf?

As you saw, there are many pre-trained embedding models available online. Let's try using one of these on this data. 

Zeugma is a package where you can use embeddings in sklearn.
https://github.com/nkthiebaut/zeugma

It allows you to directly download pre-trained models that have been released from the gensim website.


Let's use a basic glove model.

In [ ]:
glove = EmbeddingTransformer('glove')


We now need to transform our training data to map to this embedding model.

In [ ]:
glove_transformed_training_data = glove.transform(trainingdata['txt'])

Have a look at what the data now looks like with this representation

In [ ]:
glove_transformed_training_data

Let's build a classifier with this representation and evaluate with 10-fold cross-validation.

In [ ]:
kneighbour_classifier = KNeighborsClassifier().fit(glove_transformed_training_data, trainingdata['label'])
scoring = ['precision_macro', 'recall_macro','precision_micro','recall_micro', 'f1_micro', 'f1_macro']
scores = cross_validate(kneighbour_classifier, glove_transformed_training_data, trainingdata['label'], scoring=scoring, cv=10, return_train_score=False)
scoresdf = pd.DataFrame(scores)
scoring = ['test_precision_macro', 'test_recall_macro','test_precision_micro','test_recall_micro', 'test_f1_micro', 'test_f1_macro']
bp = scoresdf.boxplot(column=scoring, grid=False, rot=45,)
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('K nearest neighbour, pre-trained embeddings')
plt.show()

What do you think about these results? What is happening here?

There are a number of pre-trained embedding models that have been released openly, a full list of pretrained embeddings to experiment with using this library: https://github.com/RaRe-Technologies/gensim-data#models

You can play with these if you want, e.g.:
glove = EmbeddingTransformer('glove-wiki-gigaword-50')

# 8: Classifiers, representations, evaluation

You've now seen that you get very different results depending on which representation you use, which classifier, and also that there are many different metrics to analyse.

Let's try some different configurations all in one go. We'll create a dictionary with the three different types of representations, and a list of different classification algorithms, and apply all these configurations to see what seems to yield best results according to a chosen evaluation metric using 10-fold cross validation.

In [ ]:

representations = {}

vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=None,
                             tokenizer=word_tokenize, max_features=500)
xtrain_countvect = vectorizer.fit_transform(trainingdata['txt'])
representations['CountVectorizer'] = xtrain_countvect

tfidf_vect = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopWords)
tfidf_vect.fit(trainingdata['txt'])
xtrain_tfidf =  tfidf_vect.transform(trainingdata['txt'])
representations['TfidfVectorizer'] = xtrain_tfidf

x_train_glove = glove.transform(trainingdata['txt'])
representations['pretrained_glove'] = x_train_glove



CV = 10

classifier_models = [
        RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
        DecisionTreeClassifier(),
        SVC(),
        LinearSVC(multi_class='ovr', C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      penalty='l2', random_state=0, tol=1e-05, verbose=0),
        SGDClassifier(),
        LogisticRegression(random_state=0),
        KNeighborsClassifier(),
]

cv_df = pd.DataFrame(index=range(CV * (len(classifier_models)*len(representations))))
entries = []


for representation, transformed_vector in representations.items():
    score = 'f1_micro'
    for model in classifier_models:
      model_name = model.__class__.__name__+'_'+representation
      accuracies = cross_val_score(model, transformed_vector, trainingdata['label'], scoring=score, cv=CV)
      for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', score])
bp = cv_df.boxplot(by='model_name', column=[score], grid=False, rot=90, figsize=(12,8))
[ax_tmp.set_xlabel('') for ax_tmp in np.asarray(bp).reshape(-1)]
fig = np.asarray(bp).reshape(-1)[0].get_figure()
fig.suptitle('10-fold cross validation results')
plt.show()

What conclusions do you draw from this? Which classifier and which representation would you choose as your final model? Why?

# Assignment 1: Your turn to build a classifier

Answer questions by placing your code in the code cell below, running it to provide output, and giving your written answers in this markdown cell

**1.1 Choose one classifier and one representation format and test it on the blind test data. What do you think about your results, and how do they relate to the 10-fold cross validation results above?** 

**1.2 What other configurations could you try before deciding on a final model?**

**1.3 Is it appropriate to experiment with this on the test data? Why or why not?** 

In [ ]:
## PUT YOUR CODE ANSWERS IN HERE, AND RUN IT TO GIVE OUTPUT
## First step: Transform your training and test data to your chosen representation. 

## choose a representation: CountVectorizer or TfidfVectorizer, or embeddings


## transform the training data 
# something like this: transformed_training_data = chosen_representation.transform(trainingdata['txt'])

## transform the test data
# something like this: transformed_test_data = chosen_representation.transform(testdata['txt'])

## Second step: Create a classifier - the one you think gave best results when experimenting with cross-validation

## train the classifier on the training data
# e.g. chosen_classifier.fit(transformed_training_data, trainingdata['label'])

## predict labels on the test data
# e.g. predicted = chosen_classifier.predict(transformed_test_data)

## what results do you get?
# e.g. print(metrics.classification_report(testdata['label'], predicted, target_names=set(testdata['label'].tolist())))

# Assignment 2: use your classifier to predict labels on unseen text

Answer questions by placing your code in the three code cells below, running it to provide output, and giving your written answers in this markdown cell

**What happens if you try to predict a label with a completely new text using your chosen trained classifier model? Does it seem to classify correctly?**

In [ ]:
new_text = 'Patient with severe depression.'

## code will be something like this:
# testX = chosen_representation.transform([new_text])
# predicted = chosen_classifier.predict(testX)
# print(predicted)

In [ ]:
new_text = '5-year old girl with asthma.'

In [ ]:
new_text = 'Her pain is severe.'

# Assignment 3: Evaluation

**Write ten example sentences or paragraphs where you assign the correct label to each of them. Then pass them to the classifier and calculate precision, recall and f-score, by completing the skeleton code in the cell below, and running it to give output. Analyse and discuss the results, giving your answer in this markdown cell.**

***Gold label values: psychiatrypsychology, hematology, pain, pediatrics***

In [ ]:
new_text = []
new_gold_labels = ['pain', 'pain', 'pediatrics', 'hematology', 'psychiatrypsychology', 'pediatrics', 'psychiatrypsychology', 'hematology', 'pain', 'psychiatrypsychology']
testX = chosen_representation.transform(new_text)
predicted = chosen_classifier.predict(testX)
## compare the predicted labels with the gold labels. HINT: metrics.classification_report is useful here
print(predicted)

print(metrics.classification_report(new_gold_labels, predicted, target_names=set((new_gold_labels))))


You've now experimented with different respresentations for text classification, different classification algorithms, and experimental setups.

There are many other alternative approaches available today.


For instance, there are pre-trained biomedical embeddings, trained on scientific literature, one example: https://www.aclweb.org/anthology/W16-2922.pdf
https://github.com/cambridgeltl/BioNLP-2016

There are also many online resources with additional examples and ideas, here's one example: https://towardsdatascience.com/text-classification-in-python-dd95d264c802